In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from graphcast import data_utils
import dataclasses
from graphcast import graphcast
import his_utils
import cfgrib

In [2]:
# set up dataset path 
ds_surface = "testdata/2022-01-01_surface.grib"
ds_surface_precip = "testdata/2022-01-01_tp.grib"
ds_pressure_level = "testdata/2022-01-01_pressure level.grib"

# 1. Surface Data w/o Precipitation

precipitation 제외하고 받은 6-hrly data 전처리 하는 과정

## TODO
- dimension 이름 변경
- 불필요한 coordinate 삭제
- variable 이름 변경 -> 나중에 합치고 한 번에 진행 가능

In [3]:
ds1 = xr.open_dataset(ds_surface, engine='cfgrib')
ds1

skipping variable: paramId==212 shortName='tisr'
Traceback (most recent call last):
  File "/home/hiskim1/.conda/envs/hiskim1_graphcast/lib/python3.11/site-packages/cfgrib/dataset.py", line 721, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/hiskim1/.conda/envs/hiskim1_graphcast/lib/python3.11/site-packages/cfgrib/dataset.py", line 639, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1640995200, 1641016800, 1641038400, 1641060000, 1641081600,
       1641103200, 1641124800, 1641146400, 1641168000, 1641189600,
       1641211200, 1641232800, 1641254400, 1641276000, 1641297600,
       1641319200])) new_value=Variable(dimensions=('time',), data=array([1640973600, 1641016800, 1641060000, 1641103200, 1641146400,
       1641189600, 1641232800, 1641276000]))


<xarray.Dataset> Size: 399MB
Dimensions:     (time: 16, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 8B ...
  * time        (time) datetime64[ns] 128B 2022-01-01 ... 2022-01-04T18:00:00
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  (time) datetime64[ns] 128B ...
Data variables:
    u10         (time, latitude, longitude) float32 66MB ...
    v10         (time, latitude, longitude) float32 66MB ...
    t2m         (time, latitude, longitude) float32 66MB ...
    z           (time, latitude, longitude) float32 66MB ...
    lsm         (time, latitude, longitude) float32 66MB ...
    msl         (time, latitude, longitude) float32 66MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-07-25T09:23 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
ds1 = ds1.drop(['number', 'step', 'surface', 'valid_time'])
ds1 = ds1.rename({"z" : "geopotential_at_surface"})
ds1

/tmp/ipykernel_1020551/2974761243.py:1: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds1 = ds1.drop(['number', 'step', 'surface', 'valid_time'])


<xarray.Dataset> Size: 399MB
Dimensions:                  (time: 16, latitude: 721, longitude: 1440)
Coordinates:
  * time                     (time) datetime64[ns] 128B 2022-01-01 ... 2022-0...
  * latitude                 (latitude) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * longitude                (longitude) float64 12kB 0.0 0.25 ... 359.5 359.8
Data variables:
    u10                      (time, latitude, longitude) float32 66MB ...
    v10                      (time, latitude, longitude) float32 66MB ...
    t2m                      (time, latitude, longitude) float32 66MB ...
    geopotential_at_surface  (time, latitude, longitude) float32 66MB ...
    lsm                      (time, latitude, longitude) float32 66MB ...
    msl                      (time, latitude, longitude) float32 66MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-07-25T09:23 GRIB to CDM+CF via cfgrib-0.9.1...

# 2. Precipitation Data

Accumulates 6-hour precipitation into the next time step?
or into previous step?

## TODO
- coordinate 정리
- 강수량 합치기

In [19]:
ds2 = xr.open_dataset(ds_surface_precip, engine='cfgrib')


/tmp/ipykernel_1020551/4121294944.py:3: FutureWarning: updating coordinate 'new_time' with a PandasMultiIndex would leave the multi-index level coordinates ['time', 'step'] in an inconsistent state. This will raise an error in the future. Use `.drop_vars(['new_time', 'time', 'step'])` before assigning new coordinate values.
  ds2 = ds2.assign_coords(new_time=ds2.valid_time.values)
/tmp/ipykernel_1020551/4121294944.py:5: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds2 = ds2.drop(['number', 'surface'])


In [25]:
ds2 = ds2.stack(new_time=['time', 'step'])
ds2 = ds2.assign_coords(new_time=ds2.valid_time.values)
ds2 = ds2.rename({'new_time': 'time'})
ds2 = ds2.drop(['number', 'surface'])
ds2 = ds2.resample(time='6h').sum().isel(time=slice(1, 17))

<xarray.Dataset> Size: 12B
Dimensions:  ()
Coordinates:
    time     datetime64[ns] 8B 2022-01-01T18:00:00
Data variables:
    tp       float32 4B 0.000614

# 3. Pressure Level Data

37 level data를 처리하는 과정. 17 level이어도 동일한 방식으로 작동하도록 최대한 해보자

## TODO
- coordinate 정리

In [ ]:
ds3 = xr.open_dataset(ds_pressure_level, engine='cfgrib')
ds3

In [ ]:
ds3 = ds3.drop(['number', 'step', 'valid_time'])
ds3 = ds3.rename({"isobaricInhPa" : "level"})
ds3 = ds3.sortby('level', ascending=True)

level = ds3.level.values
level = level.astype(np.int32)

ds3 = ds3.assign_coords(level = ('level', level))
ds3

# 4. 3개의 데이터셋을 하나로 합성하기

*ds1*, *ds2*, *ds3*를 하나로 합치고 GC에 잘 들어가도록 다듬어주기

## TODO
- 한 장씩 합치는 게 좋을지, 여러 장 한 번에 합치는 게 좋을지 for memory efficiency
- 합쳐서 GC에 잘 들어가는 지까지 확인

In [ ]:
ds1 = xr.merge([ds1, ds2, ds3])

ds1 = his_utils.transform_dataset(ds1).copy()

ds1['geopotential_at_surface'] = ds1['geopotential_at_surface'].squeeze('batch')
ds1['land_sea_mask'] = ds1['land_sea_mask'].squeeze('batch')
ds1 = ds1.reindex(lat=ds1.lat[::-1]).isel(time=slice(0,14))

## 4-1. 파일 명 정하기

In [ ]:
# ds1.to_netcdf('testdata/2022-01-01.nc')

# 5. Output: GC로 준비 갈 완료!

In [ ]:
ds1